# Splitpea - Sample-specific Mode

**Goal**: Use Splitpea to generate both patient-specific rewired networks for individual Uterine carcinosarcoma (UCS) cancer samples against normal uterus tissue samples, as well a consensus network of PPI rewiring events across all UCS cancer samples.



In [ ]:
#@title 0. Install and import dependencies

%%capture
!pip install --index-url https://test.pypi.org/simple/ \
            --extra-index-url https://pypi.org/simple splitpea

!sudo apt-get install tabix  # Tabix is used under-the-hood for some of the functions

import splitpea

In [ ]:
!splitpea --help  # Splitpea can be run both as a command-line tool and as a Python library

usage: splitpea [-h]
                {run,plot,stats,calculate_delta_psi,combine_spliced_exon,preprocess_pooled,get_consensus_network}
                ...

splitpea: SPLicing InTeractions PErsonAlized

positional arguments:
  {run,plot,stats,calculate_delta_psi,combine_spliced_exon,preprocess_pooled,get_consensus_network}
    run                 calculate and output rewired network
    plot                load a saved rewired network (.edges.pickle) and
                        produce plots
    stats               compute and write gene level statistics for a rewired
                        splicing PPI network
    calculate_delta_psi
                        Calculate delta PSI values and p-values for background
                        vs compare exon comparisons
    combine_spliced_exon
                        Combine individual spliced-exon PSI files into a
                        single mean PSI file per exon
    preprocess_pooled   Download or use existing background psi files, com

In [ ]:
#@title 1. Download/importing data: splicing matrices
#@markdown We’ll fetch a UCS splicing matrix from the [IRIS](https://github.com/Xinglab/IRIS) database which contains the samples we want to compare against.
#@markdown
#@markdown **Optional:** you can also grab a normal/background matrix for Uterus, but this is redundant if you use `splitpea.preprocess_pooled`'s `background` parameter, which can auto-download the GTEx tissue matrix for you from IRIS.

!wget https://xinglabtrackhub.research.chop.edu/iris/IRIS_data.v2.0.0/db/TCGA_UCS_T/splicing_matrix/splicing_matrix.SE.cov10.TCGA_UCS_T.txt # cancer samples
print("Downloaded compare cancer matrix: /content/splicing_matrix.SE.cov10.TCGA_UCS_T.txt")

also_download_background = False  #@param {type:"boolean"}
if also_download_background:
  !wget -q https://xinglabtrackhub.research.chop.edu/iris/IRIS_data.v2.0.0/db/GTEx_Uterus/splicing_matrix/splicing_matrix.SE.cov10.GTEx_Uterus.txt # normal tissue samples
  print("Downloaded optional background normal tissue matrix: /content/splicing_matrix.SE.cov10.GTEx_Uterus.txt")

--2025-08-18 05:35:35--  https://xinglabtrackhub.research.chop.edu/iris/IRIS_data.v2.0.0/db/TCGA_UCS_T/splicing_matrix/splicing_matrix.SE.cov10.TCGA_UCS_T.txt
Resolving xinglabtrackhub.research.chop.edu (xinglabtrackhub.research.chop.edu)... 159.14.197.33
Connecting to xinglabtrackhub.research.chop.edu (xinglabtrackhub.research.chop.edu)|159.14.197.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43681277 (42M) [text/plain]
Saving to: ‘splicing_matrix.SE.cov10.TCGA_UCS_T.txt’

splicing_matrix.SE. 100%[===================>]  41.66M  1.55MB/s    in 37s     

2025-08-18 05:36:13 (1.13 MB/s) - ‘splicing_matrix.SE.cov10.TCGA_UCS_T.txt’ saved [43681277/43681277]

Downloaded compare cancer matrix: /content/splicing_matrix.SE.cov10.TCGA_UCS_T.txt


### 2. Run the built-in preprocessing function.

This function builds the inputs (`{sample}-psi.txt`) for Splitpea's sample-specific mode in the output directory (`out_psi_dir = "/content/UCS_PSI"`). It first averages exon-level splicing data across normal uterus samples to create a background, then computes ΔPSI and empirical p-values for each cancer sample (`compare_path`).

Two options are available for generating these outputs. Option A uses the `background` parameter to automatically download normal uterus samples from IRIS. Those files are saved under (`background_download_root = "/content/IRIS_uterus"`). Supported background tissues include: AdiposeTissue, AdrenalGland, Bladder, Blood, BloodVessel, Brain, Breast, CervixUteri, Colon, Esophagus, FallopianTube, Heart, Kidney, Liver, Lung, Muscle, Nerve, Ovary, Pancreas, Pituitary, Prostate, SalivaryGland, Skin, SmallIntestine, Spleen, Stomach, Testis, Thyroid, Uterus, and Vagina. Option B uses a local background path (see dropdown below).

In [ ]:
# OPTION A: auto-download GTEx tissue background from IRIS
splitpea.preprocess_pooled(
    compare_path="/content/splicing_matrix.SE.cov10.TCGA_UCS_T.txt",
    background="uterus",
    background_download_root="/content/IRIS_uterus/",
    out_psi_dir="/content/UCS_PSI"
)

Y. Pan, J.W. Phillips, B.D. Zhang, M. Noguchi, E. Kutschera, J. McLaughlin, P.A. Nesterenko, Z. Mao, N.J. Bangayan, R. Wang, W. Tran, H.T. Yang, Y. Wang, Y. Xu, M.B. Obusan, D. Cheng, A.H. Lee, K.E. Kadash-Edmondson, A. Champhekar, C. Puig-Saus, A. Ribas, R.M. Prins, C.S. Seet, G.M. Crooks, O.N. Witte, & Y. Xing, IRIS: Discovery of cancer immunotherapy targets arising from pre-mRNA alternative splicing, Proc. Natl. Acad. Sci. U.S.A. 120 (21) e2221116120, https://doi.org/10.1073/pnas.2221116120 (2023).
processing file: splicing_matrix.SE.cov10.GTEx_Uterus.txt
Starting delta PSI calculation...
Processing sample TCGA-N5-A4RM-01A-11R-A28V-07.aln...
Processing sample TCGA-N5-A4RF-01A-11R-A28V-07.aln...
Processing sample TCGA-N5-A4RU-01A-31R-A28V-07.aln...
Processing sample TCGA-NA-A4QY-01A-11R-A28V-07.aln...
Processing sample TCGA-NF-A5CP-01A-12R-A28V-07.aln...
Processing sample TCGA-ND-A4W6-01A-11R-A28V-07.aln...
Processing sample TCGA-QM-A5NM-01A-11R-A28V-07.aln...
Processing sample TCGA-

'/content/UCS_PSI'

<details>
<summary>OPTION B: to use local background path instead, replace with and run this code.</summary>

```python
splitpea.preprocess_pooled(
    compare_path="/content/splicing_matrix.SE.cov10.TCGA_UCS_T.txt",
    background_path="/content/splicing_matrix.SE.cov10.GTEx_Uterus.txt",
    out_psi_dir="/content/UCS_PSI"
)
```

</details>

In [ ]:
#@title 3. Run Splitpea (build sample specific rewired networks in parallel)
#@markdown Each `{sample}-psi.txt` file in the `input_dir = /content/UCS_PSI/` generated by `splitpea.preprocess_pooled` is used as an `input_file`.
#@markdown
#@markdown This code block simply runs:
#@markdown
#@markdown `splitpea.run(input_file, output_dir)`
#@markdown
#@markdown for each sample in parallel.
#@markdown Splitpea generates:
#@markdown `<output_dir>{sample}.edges.dat` and
#@markdown `<output_dir>{sample}.edges.pickle`,
#@markdown which are the rewired network files for that sample.
#@markdown #### Options
input_dir   = "/content/UCS_PSI/"        #@param {type:"string"}
output_dir = "/content/rewired_networks/"  #@param {type:"string"}
max_workers   = 3                       #@param {type:"integer"}
verbose = True                   #@param {type:"boolean"}

import logging

if verbose:
  logging.basicConfig(level=logging.INFO, force=True)
else:
  logging.basicConfig(level=logging.WARNING, force=True)


import os
from concurrent.futures import ProcessPoolExecutor, as_completed

os.makedirs(output_dir, exist_ok=True)

def run_one(input_file: str, output_prefix: str) -> str:
    import splitpea
    splitpea.run(input_file, output_prefix)
    return output_prefix

jobs = []
for filename in os.listdir(input_dir):
    if filename.endswith(".txt"):
        ipref = filename.replace("-psi.txt", "")
        input_file = os.path.join(input_dir, filename)
        output_prefix = os.path.join(output_dir, ipref)
        jobs.append((input_file, output_prefix))

print(f"Submitting {len(jobs)} jobs with up to {max_workers} workers...")

futures = {}
with ProcessPoolExecutor(max_workers=max_workers) as ex:
    for inp, out in jobs:
        futures[ex.submit(run_one, inp, out)] = (inp, out)

    for fut in as_completed(futures):
        inp, out = futures[fut]
        try:
            fut.result()
            print(f"Done: {inp} -> {out}")
        except Exception as e:
            print(f"Failed: {inp} -> {out} ({e})")


Submitting 57 jobs with up to 3 workers...


INFO:diff_exon:Loading DDIs....
INFO:diff_exon:Loading DDIs....
INFO:diff_exon:Loading DDIs....
INFO:diff_exon:# pfams: 8602, # ddis: 17450
INFO:diff_exon:Loading PPIs....
INFO:diff_exon:# pfams: 8602, # ddis: 17450
INFO:diff_exon:Loading PPIs....
INFO:diff_exon:# pfams: 8602, # ddis: 17450
INFO:diff_exon:Loading PPIs....
INFO:diff_exon:# proteins: 20286, # interactions: 793078
INFO:diff_exon:Loading gene-protein domain info....
INFO:diff_exon:# proteins: 20286, # interactions: 793078
INFO:diff_exon:Loading gene-protein domain info....
INFO:diff_exon:Reading differential exon results....
INFO:diff_exon:Reading in input exons...
INFO:diff_exon:# proteins: 20286, # interactions: 793078
INFO:diff_exon:Loading gene-protein domain info....
INFO:diff_exon:# of differentially expressed exons: 3173
INFO:diff_exon:Calculating network....
INFO:diff_exon:Reading differential exon results....
INFO:diff_exon:Reading in input exons...
INFO:diff_exon:# of differentially expressed exons: 3640
INFO:dif

Done: /content/UCS_PSI/TCGA-N7-A59B-01A-11R-A28V-07.aln-psi.txt -> /content/rewired_networks/TCGA-N7-A59B-01A-11R-A28V-07.aln


INFO:diff_exon:# nodes: 3647, # edges: 6613
INFO:diff_exon:Outputting as pickle...
INFO:diff_exon:Done
INFO:diff_exon:Loading DDIs....


Done: /content/UCS_PSI/TCGA-N5-A59E-01A-11R-A28V-07.aln-psi.txt -> /content/rewired_networks/TCGA-N5-A59E-01A-11R-A28V-07.aln


...

Done: /content/UCS_PSI/TCGA-N7-A4Y0-01A-12R-A28V-07.aln-psi.txt -> /content/rewired_networks/TCGA-N7-A4Y0-01A-12R-A28V-07.aln


INFO:diff_exon:# nodes: 4050, # edges: 7521
INFO:diff_exon:Outputting as pickle...
INFO:diff_exon:Done


Done: /content/UCS_PSI/TCGA-QM-A5NM-01A-11R-A28V-07.aln-psi.txt -> /content/rewired_networks/TCGA-QM-A5NM-01A-11R-A28V-07.aln


INFO:diff_exon:# nodes: 3687, # edges: 6485
INFO:diff_exon:Outputting as pickle...
INFO:diff_exon:Done


Done: /content/UCS_PSI/TCGA-N7-A4Y8-01A-11R-A28V-07.aln-psi.txt -> /content/rewired_networks/TCGA-N7-A4Y8-01A-11R-A28V-07.aln


<details>
<summary><strong>Example code snippet to run sequentially</strong></summary>

```python
import os, splitpea
data_dir = "/content/UCS_PSI/"
output_dir = "/content/rewired_networks/"
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(data_dir):
    if filename.endswith("-psi.txt") or filename.endswith(".txt"):
        ipref = filename.replace("-psi.txt", "").replace(".txt","")
        input_file = os.path.join(data_dir, filename)
        output_prefix = os.path.join(output_dir, ipref)
        print(f"Running splitpea on {input_file} -> {output_prefix}")
        splitpea.run(input_file, output_prefix)
```
</details>

### 4. (Optional) Create a consensus network

We have generated sample-level networks. To obtain a consensus network for UCS, we can run `splitpea.get_consensus_network` on the rewired network outputs. This creates two summary networks across all patient samples: one for consensus losses (`consensus_network_neg.pickle`) and one for consensus gains (`consensus_network_pos.pickle`).

In [ ]:
cons_neg, cons_pos = splitpea.get_consensus_network("/content/rewired_networks/") # pass in the path to the folder of rewired networks

[06:46:30] Loading networks...
[06:46:30] Adding /content/rewired_networks/TCGA-ND-A4WC-01A-21R-A28V-07.aln.edges.pickle....
[06:46:31] Adding /content/rewired_networks/TCGA-N8-A4PM-01A-11R-A28V-07.aln.edges.pickle....
[06:46:31] Adding /content/rewired_networks/TCGA-NA-A4QX-01A-11R-A28V-07.aln.edges.pickle....
[06:46:32] Adding /content/rewired_networks/TCGA-N7-A59B-01A-11R-A28V-07.aln.edges.pickle....
[06:46:32] Adding /content/rewired_networks/TCGA-N5-A59F-01A-11R-A28V-07.aln.edges.pickle....
[06:46:33] Adding /content/rewired_networks/TCGA-NA-A4R0-01A-11R-A28V-07.aln.edges.pickle....
[06:46:34] Adding /content/rewired_networks/TCGA-ND-A4W6-01A-11R-A28V-07.aln.edges.pickle....
[06:46:35] Adding /content/rewired_networks/TCGA-N5-A4RU-01A-31R-A28V-07.aln.edges.pickle....
[06:46:36] Adding /content/rewired_networks/TCGA-NA-A4QV-01A-11R-A28V-07.aln.edges.pickle....
[06:46:37] Adding /content/rewired_networks/TCGA-N8-A4PQ-01A-11R-A28V-07.aln.edges.pickle....
[06:46:38] Adding /content/re

### 5. Questions? Feedback?

Please submit an issue through our [Github](https://github.com/ylaboratory/splitpea_package/issues) page.

#### PyPi repository

Want to try out Splitpea? Please download it from our PyPi repository: ...

#### Citations

...